<a href="https://colab.research.google.com/github/Khushboo1990-oss/1st-Mern-project/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
# Load the datasets
X_train = pd.read_csv('/content/drive/MyDrive/X_Train_Data_Input.csv')
Y_train = pd.read_csv('/content/drive/MyDrive/Y_Train_Data_Target.csv')
X_test = pd.read_csv('/content/drive/MyDrive/X_Test_Data_Input.csv')
Y_test = pd.read_csv('/content/drive/MyDrive/Y_Test_Data_Target.csv')




In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Drop the ID columns (assuming 'ID' is the first column)
X_train.drop(columns=['ID','Column9'], inplace=True)
Y_train.drop(columns=['ID'], inplace=True)
X_test.drop(columns=['ID','Column9'], inplace=True)
Y_test.drop(columns=['ID'], inplace=True)

In [6]:
# Select multiple columns for mode calculation
mode_values = X_train[['Column0','Column4','Column3', 'Column14', 'Column15']].mode().iloc[0]  # Get mode for each column

# Fill missing values with the mode for each corresponding column
X_train[['Column0','Column4','Column3', 'Column14', 'Column15']] = X_train[['Column0','Column4','Column3', 'Column14', 'Column15']].fillna(mode_values)

 # Select multiple columns for mode calculation
mode_values = X_test[['Column0','Column4','Column3', 'Column14', 'Column15']].mode().iloc[0]  # Get mode for each column

# Fill missing values with the mode for each corresponding column
X_test[['Column0','Column4','Column3', 'Column14', 'Column15']] = X_test[['Column0','Column4','Column3', 'Column14', 'Column15']].fillna(mode_values)

In [7]:
# Select multiple columns for median calculation
median_values = X_train[['Column5', 'Column8']].median()  # Get median for each column

# Fill missing values with the median for each corresponding column
X_train[['Column5', 'Column8']] = X_train[['Column5', 'Column8']].fillna(median_values)

# Select multiple columns for median calculation
median_values = X_test[['Column5', 'Column8']].median()  # Get median for each column

# Fill missing values with the median for each corresponding column
X_test[['Column5', 'Column8']] = X_test[['Column5', 'Column8']].fillna(median_values)

In [8]:
X_train.fillna({'Column6': X_test['Column6'].mean()}, inplace=True)
X_test.fillna({'Column6': X_test['Column6'].mean()}, inplace=True)

In [9]:
scaler = StandardScaler()

# Standard scale 'Column1' after handling non-numeric data
X_train['Column1'] = scaler.fit_transform(X_train[['Column1']])
X_test['Column1'] = scaler.fit_transform(X_test[['Column1']])
X_train['Column2'] = scaler.fit_transform(X_train[['Column2']])
X_test['Column2'] = scaler.fit_transform(X_test[['Column2']])

In [10]:
X_train.drop(columns=['Column0','Column10','Column11','Column13','Column3','Column21'], inplace=True)


In [11]:
X_test.drop(columns=['Column0','Column10','Column11','Column13','Column3','Column21'], inplace=True)

In [12]:
X_train.drop('Column16', axis=1, inplace=True)
X_test.drop('Column16', axis=1, inplace=True)

In [ ]:
print(X_train.columns)
print(X_test.columns)

Index(['Column1', 'Column2', 'Column4', 'Column5', 'Column6', 'Column7',
       'Column8', 'Column12', 'Column14', 'Column15', 'Column17', 'Column18',
       'Column19', 'Column20'],
      dtype='object')
Index(['Column1', 'Column2', 'Column4', 'Column5', 'Column6', 'Column7',
       'Column8', 'Column12', 'Column14', 'Column15', 'Column17', 'Column18',
       'Column19', 'Column20'],
      dtype='object')


In [ ]:
print(X_train.isna().sum())

Column1     0
Column2     0
Column4     0
Column5     0
Column6     0
Column7     0
Column8     0
Column12    0
Column14    0
Column15    0
Column17    0
Column18    0
Column19    0
Column20    0
dtype: int64


In [ ]:
print(X_test.isna().sum())

Column1     0
Column2     0
Column4     0
Column5     0
Column6     0
Column7     0
Column8     0
Column12    0
Column14    0
Column15    0
Column17    0
Column18    0
Column19    0
Column20    0
dtype: int64


In [ ]:
!pip install lightgbm
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# Convert the dataset into LightGBM dataset format
train_data = lgb.Dataset(X_train, label=Y_train)
test_data = lgb.Dataset(X_test, label=Y_test, reference=train_data)

# Define the LightGBM parameters
params = {
    'objective': 'binary',  # for binary classification
    'boosting_type': 'gbdt',  # Gradient Boosting Decision Tree
    'metric': 'binary_logloss',  # Evaluation metric
    'num_leaves': 31,  # Number of leaves in one tree
    'learning_rate': 0.05,  # Step size for shrinking the contribution of each tree
    'feature_fraction': 0.9  # Fraction of features to be used
}

# Train the model with early stopping using callbacks
gbm = lgb.train(
    params,
    train_data,
    num_boost_round=100,
    valid_sets=[test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# Make predictions
y_pred_prob = gbm.predict(X_test, num_iteration=gbm.best_iteration)  # Probability estimates
y_pred = [1 if x > 0.5 else 0 for x in y_pred_prob]  # Convert probabilities to binary predictions

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.0548728
Accuracy: 97.72%


In [ ]:
report = classification_report(Y_test, Y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.98      0.99    237034
           1       0.82      0.93      0.87     24678

    accuracy                           0.97    261712
   macro avg       0.91      0.95      0.93    261712
weighted avg       0.98      0.97      0.98    261712



In [84]:
#Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

# Train the model
rf_classifier.fit(X_train, Y_train)

# Predictions
Y_pred = rf_classifier.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 0.9727372073118542
              precision    recall  f1-score   support

           0       0.99      0.98      0.98    237034
           1       0.83      0.89      0.86     24678

    accuracy                           0.97    261712
   macro avg       0.91      0.93      0.92    261712
weighted avg       0.97      0.97      0.97    261712



In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import precision_score, accuracy_score

# Convert the dataset into LightGBM Dataset format
train_data_selected = lgb.Dataset(X_train, label=Y_train)
test_data_selected = lgb.Dataset(X_test, label=Y_test, reference=train_data_selected)

params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'lambda_l1': 0.2,
    'lambda_l2': 0.2,
    'feature_fraction': 0.8,
    'min_data_in_leaf': 50,
}

# Cross-validation with Stratified K-Folds
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation precision evaluation
model = lgb.LGBMClassifier(**params)
precision_scores = cross_val_score(model, X_train, Y_train.values.ravel(), cv=kf, scoring='precision')  # or Y_train.to_numpy().ravel()
print(f'Mean Precision (Cross-validated): {precision_scores.mean():.4f}')


# Train the final model on the entire training set
final_model = lgb.LGBMClassifier(**params)
final_model.fit(X_train, Y_train)

# Predict probabilities for test set
y_pred_prob = final_model.predict_proba(X_test)[:, 1]

# Adjust the decision threshold to increase precision
threshold = 0.6  # You can experiment with this threshold to get better precision
y_pred = [1 if prob > threshold else 0 for prob in y_pred_prob]

# Calculate precision and accuracy
precision = precision_score(Y_test, y_pred)
accuracy = accuracy_score(Y_test, y_pred)

print(f'Precision: {precision:.4f}')
print(f'Accuracy: {accuracy:.4f}')


In [18]:
print("Accuracy:", accuracy_score(Y_test, y_pred))
print(classification_report(Y_test, y_pred))

Accuracy: 0.9247799107415785
              precision    recall  f1-score   support

           0       0.92      1.00      0.96    237034
           1       0.96      0.21      0.34     24678

    accuracy                           0.92    261712
   macro avg       0.94      0.60      0.65    261712
weighted avg       0.93      0.92      0.90    261712



In [62]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Convert the dataset into LightGBM Dataset format
train_data_selected = lgb.Dataset(X_train, label=Y_train)
test_data_selected = lgb.Dataset(X_test, label=Y_test, reference=train_data_selected)

params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'num_leaves': 30,
    'learning_rate': 0.01,
    'lambda_l1': 0.15,
    'lambda_l2': 0.2,
    'feature_fraction': 0.8,
    'min_data_in_leaf': 40,
    'scale_pos_weight':4, # Adjust this to handle class imbalance
    'verbosity':1
}

# Cross-validation with Stratified K-Folds
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation precision and recall evaluation
model = lgb.LGBMClassifier(**params)

# Using cross_val_score for both precision and recall
precision_scores = cross_val_score(model, X_train, Y_train.values.ravel(), cv=kf, scoring='precision')
recall_scores = cross_val_score(model, X_train, Y_train.values.ravel(), cv=kf, scoring='recall')

print(f'Mean Precision (Cross-validated): {precision_scores.mean():.4f}')
print(f'Mean Recall (Cross-validated): {recall_scores.mean():.4f}')

# Train the final model on the entire training set
final_model = lgb.LGBMClassifier(**params)
final_model.fit(X_train, Y_train)

# Predict probabilities for test set
y_pred_prob = final_model.predict_proba(X_test)[:, 1]
# Fix the threshold to a specific value (e.g., 0.5)
threshold = 0.55  # Set threshold value here
y_pred = [1 if prob > threshold else 0 for prob in y_pred_prob]

# Calculate precision, recall, and F1 score
precision = precision_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)

print(f'Threshold: {threshold:.1f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')


[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2
[LightGBM] [Warning] lambda_l1 is set=0.15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.15
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2
[LightGBM] [Warning] lambda_l1 is set=0.15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.15
[LightGBM] [Info] Number of positive: 59226, number of negative: 568880
[LightGBM] [Info] 

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2
[LightGBM] [Warning] lambda_l1 is set=0.15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.15
[LightGBM] [Info] Number of positive: 74033, number of negative: 711100
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.138621 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2193
[LightGBM] [Info] Number of data points in the train set: 785133, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.094294 -> initscore=-2.262302
[LightGBM] [Info] Start training from score -2.262302
[LightGBM] [Warning] min_data_in_leaf is set=40, 

In [61]:
report = classification_report(Y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.98      0.99    237034
           1       0.85      0.89      0.87     24678

    accuracy                           0.98    261712
   macro avg       0.92      0.94      0.93    261712
weighted avg       0.98      0.98      0.98    261712



In [45]:
report = classification_report(Y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98    237034
           1       0.90      0.72      0.80     24678

    accuracy                           0.97    261712
   macro avg       0.94      0.86      0.89    261712
weighted avg       0.96      0.97      0.96    261712



In [ ]:
from sklearn.model_selection import cross_val_score

# Validate the model using cross-validation
scores = cross_val_score(model, X_train, Y_train, cv=5, scoring='accuracy')
print(f'Mean cross-validated accuracy: {scores.mean():.4f}')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier

# Define the model
model = LGBMClassifier()

param_dist = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.1, 0.05],
    'lambda_l1': [0.0, 0.1, 0.2],
    'lambda_l2': [0.0, 0.1, 0.2],
    'feature_fraction': [0.8, 0.9, 1.0],
    'min_data_in_leaf': [20, 50, 100]
}

random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=50, cv=5, scoring='accuracy')
random_search.fit(X_train, Y_train)

print(f'Best Parameters: {random_search.best_params_}')


In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import precision_score, accuracy_score

# Define parameters for LightGBM
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'num_leaves': 20,  # Reducing complexity
    'learning_rate': 0.05,  # Trying a slightly higher learning rate
    'lambda_l1': 0.2,
    'lambda_l2': 0.2,
    'feature_fraction': 0.8,
    'min_data_in_leaf': 100,  # Reducing overfitting
    'scale_pos_weight': 10,  # Adjusted for class imbalance
    # 'early_stopping_round': 30,  # Shortened early stopping to avoid overfitting
    'verbose': -1
}

# Prepare LightGBM dataset for training
train_data_selected = lgb.Dataset(X_train, label=Y_train)
valid_data_selected = lgb.Dataset(X_test, label=Y_test)

# Train LightGBM model with early stopping
gbm = lgb.train(params,
                train_data_selected,
                num_boost_round=1000,
                valid_sets=[valid_data_selected],
                valid_names=['validation'],
                callbacks=[lgb.early_stopping(stopping_rounds=30)])

# Predict probabilities for test set
y_pred_prob = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Adjust the decision threshold to increase precision
threshold = 0.7  # Increasing the threshold to reduce false positives
y_pred = [1 if prob > threshold else 0 for prob in y_pred_prob]

# Calculate precision and accuracy on the test set
precision = precision_score(Y_test, y_pred)
accuracy = accuracy_score(Y_test, y_pred)

print(f'Precision: {precision:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# # Feature importance (optional)
# importance_df = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': gbm.feature_importance(importance_type='gain')
# })
# print(importance_df.sort_values(by='Importance', ascending=False))

# Cross-validation precision evaluation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = lgb.LGBMClassifier(**params)

# Evaluate cross-validated precision
precision_scores = cross_val_score(model, X_train, Y_train.values.ravel(), cv=kf, scoring='precision')
print(f'Mean Precision (Cross-validated): {precision_scores.mean():.4f}')




In [ ]:
report = classification_report(Y_test, Y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.98      0.99    237034
           1       0.82      0.93      0.87     24678

    accuracy                           0.97    261712
   macro avg       0.91      0.95      0.93    261712
weighted avg       0.98      0.97      0.98    261712



In [ ]:
# Convert Y_train to 1D array if it's a single column DataFrame
precision_scores = cross_val_score(model, X_train, Y_train.values.ravel(), cv=kf, scoring='precision')

print(f'Mean Precision (Cross-validated): {precision_scores.mean():.4f}')

Mean Precision (Cross-validated): 0.7590
